# mwb_flow.prep Package Example 1.1
This example is update to the prep_Example1_PullData.ipynd and serves three purposes. First, it shows a workflow for creating a dataset of monthly precip and temperature data that can be used as an input for the mwb_flow model. Second, it compares three methods of retriving GridMet data and calculating zonal stats for each polygon. Lastly, it contains code for each proposed method that can be refrenced if needed later.  

In [1]:
# More inports than nessasary...
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr

# Block of imports needed for GRIDtool.grid_area_weighted_volume
import rasterio as rio
import pandas as pd
from chmdata.thredds import GridMet, BBox
import numpy as np
from shapely.geometry import Polygon



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
from prep.metdata import get_gridmet_at_points, get_gridmet_for_polygons, get_gridmet_for_polygons_with_xvec
from prep.datafile import check_format

import py3dep
from tqdm import tqdm
from prep.utils import get_gridmet_cells
from config import GRIDMET_PARAMS
import GRIDtools as gt
import rioxarray
import xvec 
import shapely
from geocube.api.core import make_geocube



Initializing mwb_flow.prep module.


## Load in Shape File
Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column with the gage station numbers was used. 

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
exres = exres.to_crs(4326) # This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.

## Pull GridMet Data and Perform Zonal Stats
Three functions from prep.metdata.py are shown below. The first two functions, 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_xvec()' pull meterologic data and perform area zonal stats (area weighted for precip and mean for temperature parameters) for polygons but use diffrent packages to do so. More details for each function below.

The last function 'get_gridmet_at_points()' shows the original function that pulls meterologic data but only calculates the mean for polygons. The function is slow. 

### get_gridmet_for_polygons()

This function uses the geocube package which rasterizes all of the polygons at once. <ins>This function appears to be the fastest so we will use it for the rest of the workflow.</ins>

This range of dates was used since there are no data-gaps in the discharge record for each location. 

In [3]:
exres_met = get_gridmet_for_polygons(exres, "gageID", start='2016-01-01', end='2019-12-31')
exres_met

<xarray.Dataset> Size: 222kB
Dimensions:        (location: 6, time: 1461)
Coordinates:
  * location       (location) int64 48B 1000 2800 3000 5500 9500 9600
  * time           (time) datetime64[ns] 12kB 2016-01-01 ... 2019-12-31
    lat            (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
    area           (location) float64 48B 185.3 34.26 186.8 139.0 102.1 57.96
Data variables:
    max_temp       (time, location) float64 70kB 264.8 265.4 ... 270.5 270.6
    min_temp       (time, location) float64 70kB 255.3 255.9 ... 263.7 263.7
    precip_volume  (time, location) float64 70kB 2.952e+04 0.0 ... 1.773e+06
Attributes:
    featureType:  timeSeries

### get_gridmet_for_polygons_with_xvec()

This function uses the xvec package to calculate the mean of non-area weighted parameters. The iterate method argument was used to perform zonal statistics which is a slower method than the 'rasterize' method in the package. This option has the potential to be fast if bugs in the 'rasterize' method are fixed in the package.

In [4]:
# get_gridmet_for_polygons_with_xvec(exres, "gageID", start='2016-08-01', end='2016-11-30')

### get_gridmet_at_points()
This function does not calculate the area weighted volume for precipitation that the other two function do but we can run this function just to comapre the speed with the other two fuctions.

In [5]:
# get_gridmet_at_points(exres, 'gageID', start='2016-08-01', end='2016-11-30')

## Format Dataset to Use with MWB_Flow model

Calculate the monthly average temperature.

In [6]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

<xarray.DataArray 'mo_temp' (time: 48, location: 6)> Size: 2kB
array([[-3.11687345, -2.70322581, -2.73560794, -3.3437276 , -2.56693548,
        -2.88924731],
       [ 0.35994695,  0.90344828,  1.17665782,  0.08659004,  1.40344828,
         1.47931034],
       [ 1.1573201 ,  1.73924731,  2.22133995,  0.66899642,  2.19455645,
         2.39032258],
       [ 6.80705128,  7.56611111,  7.94141026,  6.20481481,  7.79020833,
         7.83777778],
       [ 8.4792804 ,  9.15268817,  9.80260546,  7.76397849,  9.3641129 ,
         9.51989247],
       [13.3074359 , 14.21166667, 14.69820513, 12.97666667, 14.78854167,
        14.88      ],
       [14.51488834, 15.49086022, 15.908933  , 14.40143369, 16.3078629 ,
        16.44247312],
       [14.69875931, 15.77043011, 16.00545906, 14.66577061, 16.36350806,
        16.41451613],
       [ 9.21307692,  9.93722222, 10.43320513,  9.01148148, 10.47208333,
        10.66777778],
       [ 4.00942928,  4.52258065,  4.98722084,  3.61129032,  4.98568548,
         5.21397849],
...
       [-1.98598015, -1.25752688, -1.23238213, -2.32939068, -1.425     ,
        -1.93602151],
       [ 3.19269231,  3.81722222,  4.35346154,  2.49518519,  4.071875  ,
         4.30111111],
       [ 8.20099256,  9.01827957,  9.53759305,  7.54157706,  9.17096774,
         9.31666667],
       [11.50435897, 12.48111111, 12.97076923, 11.18111111, 13.03854167,
        13.19777778],
       [15.08796526, 16.05591398, 16.42431762, 14.91827957, 16.62842742,
        16.74354839],
       [15.77915633, 16.77204301, 17.10657568, 15.68261649, 17.37439516,
        17.50645161],
       [ 9.93461538, 10.63666667, 11.15782051,  9.60888889, 11.07520833,
        11.36111111],
       [-0.32543424,  0.33010753,  0.76116625, -0.83315412,  0.55120968,
         0.78387097],
       [-1.57833333, -1.23222222, -0.98782051, -1.74537037, -0.83375   ,
        -0.93777778],
       [-3.38188586, -2.9811828 , -2.86600496, -3.49175627, -2.36370968,
        -2.27150538]])
Coordinates:
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
    lat       (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
    area      (location) float64 48B 185.3 34.26 186.8 139.0 102.1 57.96
  * time      (time) datetime64[ns] 384B 2016-01-01 2016-02-01 ... 2019-12-01
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

Calculate the monthly precipitation.

In [7]:
# Calculate the monthly precipitation in mm
monthly_precip = exres_met.precip_volume.resample(time = "MS").sum() # summarize by monthly sum
area_m2 = (exres_met['area'].values * 1000000) # convert polygon areas from km^2 to m^2
monthly_precip = (monthly_precip / np.tile(area_m2, (len(monthly_precip['time']), 1))) * 1000 # convert precip volume (m^3) to length (m) and then to mm

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Precipitation', 'units': 'mm'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

<xarray.DataArray 'mo_precip' (time: 48, location: 6)> Size: 2kB
array([[119.3455353 , 184.59720135,  98.12175165, 111.61018854,
        152.27301236, 148.63021009],
       [ 94.63924562, 145.43858069,  74.15895668,  83.45823752,
        120.97154505, 116.10222261],
       [144.25340072, 228.31187244, 112.25430359, 130.45303859,
        187.30546387, 181.81173021],
       [ 36.25049138,  43.95722595,  31.78970888,  33.41223479,
         41.38451784,  39.02883899],
       [125.00904306, 149.36390159, 109.39459231, 117.89805702,
        147.23354913, 138.68384638],
       [ 56.67856819,  61.1393873 ,  46.18624615,  51.74221555,
         66.47796325,  61.68564098],
       [ 29.93663646,  34.12070764,  27.06835312,  26.67364808,
         33.41465677,  31.61076836],
       [ 14.3587885 ,  17.53555949,  14.49355228,  14.69665846,
         15.93423891,  15.25906078],
       [ 70.29091085,  85.14282051,  58.02108855,  64.13355572,
         83.4617455 ,  80.42941738],
       [325.51704494, 467.98438061, 270.63126237, 305.30359475,
        396.53458388, 392.90674388],
...
       [ 29.76877921,  45.46192214,  24.34269103,  26.60291725,
         38.26805618,  36.1540538 ],
       [112.83964398, 150.59310675,  96.74336043, 105.63930274,
        132.83548689, 127.23397836],
       [ 42.81464128,  41.62802027,  40.41964163,  41.63046515,
         46.53345258,  42.94139344],
       [ 68.04003512,  78.72950061,  60.48188431,  68.85394953,
         77.0464066 ,  74.27410387],
       [ 23.88386715,  24.46778326,  22.09652964,  22.08257322,
         25.85469746,  23.09934961],
       [ 30.50103873,  35.160156  ,  25.47031693,  29.42328748,
         35.16721393,  34.04746658],
       [ 84.3318177 ,  99.33903958,  79.57610229,  80.21340268,
         94.18091006,  90.97398139],
       [ 87.4289137 , 123.09103456,  69.22981722,  76.21104691,
        108.63370249, 105.4143821 ],
       [ 38.80381058,  54.61073833,  33.64370547,  37.21754909,
         46.44812082,  45.47727563],
       [165.81244653, 261.33829677, 133.96575084, 154.27952419,
        213.07500965, 210.29101869]])
Coordinates:
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
    lat       (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
    area      (location) float64 48B 185.3 34.26 186.8 139.0 102.1 57.96
  * time      (time) datetime64[ns] 384B 2016-01-01 2016-02-01 ... 2019-12-01
Attributes:
    standard_name:  Monthy Precipitation
    units:          mm

In [9]:
Monthly_Q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
Monthly_Q

<xarray.DataArray 'mo_discharge_vol' (time: 48, location: 6)> Size: 2kB
array([[2.38811921e+06,            nan,            nan,            nan,
                   nan, 5.62439685e+05],
       [4.12108321e+06,            nan,            nan,            nan,
                   nan, 6.67044475e+06],
       [6.61966276e+06,            nan,            nan,            nan,
                   nan, 1.12815370e+07],
       [1.30576234e+07,            nan,            nan,            nan,
                   nan, 2.71272525e+07],
       [1.26617420e+07,            nan,            nan,            nan,
                   nan, 3.02055135e+07],
       [5.40685404e+06, 3.36059629e+05, 6.02938767e+06, 4.37353719e+06,
                   nan, 1.62594495e+07],
       [2.87897566e+06, 2.02944257e+05, 3.72456773e+06, 4.44339405e+06,
                   nan, 4.73120155e+06],
       [1.49819010e+06, 1.09312995e+05, 1.63141022e+06, 1.61995616e+06,
        1.63690507e+06, 3.44924133e+05],
       [1.51415197e+06, 1.01599758e+05, 1.81574796e+06, 1.10916828e+06,
        3.09537017e+06, 6.65349864e+05],
       [3.10598423e+06, 2.96364145e+05, 3.91241121e+06, 3.31669810e+06,
        6.68340208e+06, 5.92622781e+06],
...
       [4.03619927e+06, 0.00000000e+00, 0.00000000e+00, 3.67549167e+06,
        0.00000000e+00, 1.00223967e+06],
       [1.71857773e+07, 9.61966491e+05, 0.00000000e+00, 1.10011860e+07,
        4.38426032e+07, 4.49011192e+07],
       [2.49594846e+07, 2.15501255e+06, 0.00000000e+00, 2.01781216e+07,
        5.79185959e+07, 6.39772421e+07],
       [1.14707328e+07, 7.78427553e+05, 9.07039952e+06, 1.38682930e+07,
        3.03770108e+07, 3.06730235e+07],
       [3.85134775e+06, 3.34115825e+05, 5.10744359e+06, 5.41594066e+06,
        1.10703858e+07, 8.99547672e+06],
       [1.89614804e+06, 6.93588367e+04, 2.83838035e+06, 1.56632621e+06,
        4.96606720e+06, 3.43767540e+06],
       [1.37171030e+06, 2.73399007e+05, 2.24867817e+06, 1.69618769e+06,
        4.52917609e+06, 2.98616880e+06],
       [2.14560139e+06, 1.52388383e+06, 2.97608636e+06, 2.69337029e+06,
        6.33974011e+06, 5.28790733e+06],
       [1.86010132e+06, 1.03055450e+06, 2.83344642e+06, 2.76456105e+06,
        5.95046955e+06, 5.62887586e+06],
       [1.60609326e+06,            nan, 2.84434286e+05, 3.13866080e+05,
        5.11356207e+06, 4.57253654e+06]])
Coordinates:
  * time      (time) datetime64[ns] 384B 2016-01-01 2016-02-01 ... 2019-12-01
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
Attributes:
    standard_name:  Monthly Discharge Volume
    units:          m^3

In [10]:
area_m2 = (exres_met['area'].values * 1000000) # convert polygon areas from km^2 to m^2
monthly_precip = (monthly_precip / np.tile(area_m2, (len(monthly_precip['time']), 1))) * 1000 # convert precip volume (m^3) to length (m) and then to mm

<xarray.DataArray 'mo_discharge_vol' (time: 48, location: 6)> Size: 2kB
array([[2.38811921e+03,            nan,            nan,            nan,
                   nan, 5.62439685e+02],
       [4.12108321e+03,            nan,            nan,            nan,
                   nan, 6.67044475e+03],
       [6.61966276e+03,            nan,            nan,            nan,
                   nan, 1.12815370e+04],
       [1.30576234e+04,            nan,            nan,            nan,
                   nan, 2.71272525e+04],
       [1.26617420e+04,            nan,            nan,            nan,
                   nan, 3.02055135e+04],
       [5.40685404e+03, 3.36059629e+02, 6.02938767e+03, 4.37353719e+03,
                   nan, 1.62594495e+04],
       [2.87897566e+03, 2.02944257e+02, 3.72456773e+03, 4.44339405e+03,
                   nan, 4.73120155e+03],
       [1.49819010e+03, 1.09312995e+02, 1.63141022e+03, 1.61995616e+03,
        1.63690507e+03, 3.44924133e+02],
       [1.51415197e+03, 1.01599758e+02, 1.81574796e+03, 1.10916828e+03,
        3.09537017e+03, 6.65349864e+02],
       [3.10598423e+03, 2.96364145e+02, 3.91241121e+03, 3.31669810e+03,
        6.68340208e+03, 5.92622781e+03],
...
       [4.03619927e+03, 0.00000000e+00, 0.00000000e+00, 3.67549167e+03,
        0.00000000e+00, 1.00223967e+03],
       [1.71857773e+04, 9.61966491e+02, 0.00000000e+00, 1.10011860e+04,
        4.38426032e+04, 4.49011192e+04],
       [2.49594846e+04, 2.15501255e+03, 0.00000000e+00, 2.01781216e+04,
        5.79185959e+04, 6.39772421e+04],
       [1.14707328e+04, 7.78427553e+02, 9.07039952e+03, 1.38682930e+04,
        3.03770108e+04, 3.06730235e+04],
       [3.85134775e+03, 3.34115825e+02, 5.10744359e+03, 5.41594066e+03,
        1.10703858e+04, 8.99547672e+03],
       [1.89614804e+03, 6.93588367e+01, 2.83838035e+03, 1.56632621e+03,
        4.96606720e+03, 3.43767540e+03],
       [1.37171030e+03, 2.73399007e+02, 2.24867817e+03, 1.69618769e+03,
        4.52917609e+03, 2.98616880e+03],
       [2.14560139e+03, 1.52388383e+03, 2.97608636e+03, 2.69337029e+03,
        6.33974011e+03, 5.28790733e+03],
       [1.86010132e+03, 1.03055450e+03, 2.83344642e+03, 2.76456105e+03,
        5.95046955e+03, 5.62887586e+03],
       [1.60609326e+03,            nan, 2.84434286e+02, 3.13866080e+02,
        5.11356207e+03, 4.57253654e+03]])
Coordinates:
  * time      (time) datetime64[ns] 384B 2016-01-01 2016-02-01 ... 2019-12-01
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600

Create an xarray.Dataset and check the format to insure everything is correct

In [ ]:
metdata_input = xr.merge([Monthly_Temp, Monthly_Precip, Monthly_Q])
check_format(metdata_input)

Here is the final dataset that will be used with the model. 

In [ ]:
metdata_input

## Examples of Code from Above Functions (for refrence only)

<ins>The following code is identical to code found in the above fuctions or of earlier approchs to pulling, calculating area weighted volumes or means of polygons. The following cells do not need to be run in this example</ins>

Below is the code used in the 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_with_xvec()' functions. They are included for testing or troubleshooting if needed. The third example shows code similar to 'get_gridmet_at_points()' but it has been modified to pull metdata and calculate area weighted volume for precipitation is shown below. It is included as an example of earlier approches.  

In [15]:
# Give arguments for upcoming functions
in_geom = exres
gdf_index_col = "gageID"
start='2016-08-01'
end='2016-11-30'
crs = 4326

### get_gridmet_for_polygons() 

In [ ]:
var_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]

    if p == 'pr':
        gmet_input = gmet_input/1000 #convert from mm to meters
        vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, gdf_index_col)
        vol_xds = vol_xds.drop('area')
    else:
        # This is done for each output of the above code
        in_geom[gdf_index_col] = in_geom[gdf_index_col].astype(int) 
        gmet_input = gmet_input.rio.write_crs(input_crs=4326).rio.clip(in_geom.geometry.values, in_geom.crs)
        gmet_input.name = p

        # Added coords previously not needed in package example
        grid_out = make_geocube(vector_data=in_geom, measurements=[gdf_index_col], like=gmet_input).set_coords(gdf_index_col)

        for date in range(0, len(gmet_input.time.values)):
            gmet_ts = gmet_input[date,:,:]
            grid_ts = grid_out
            
            grid_ts[p] = (grid_out.dims, gmet_ts.values, gmet_ts.attrs, gmet_ts.encoding)
            grid_ts = grid_out.drop("spatial_ref").groupby(grid_out[gdf_index_col]).mean()

            xda = grid_ts[p]
            xda = xda.expand_dims({"time": 1}).assign_coords(time=('time', [gmet_ts.time.values]))
            var_list.append(xda)
    xds = xr.merge(var_list)

# Add lat coordinate and make match order of xds locations
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom[gdf_index_col])
lat_df = lat_df.reindex(list(xds[gdf_index_col].values.astype(int)))
    
xds = xr.Dataset(
    {
        "max_temp": (['time', 'location'], xds["tmmx"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['time', 'location'], xds["tmmn"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "location": (['location'], xds[gdf_index_col].values.astype(int), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}), # Keep the order of xds
        "time": xds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

if 'pr' in GRIDMET_PARAMS:
    output = xr.merge([xds, vol_xds])# vol_xds reorders to match xds
else: 
    output = xds

output

### get_gridmet_for_polygons_with_xvec()

In [ ]:
gmet_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    if p == 'pr':
        vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, gdf_index_col)
        vol_xds = vol_xds.transpose('location', 'time').drop('area')
    else:
        gmet_list.append(gmet_input)
ds = xr.merge(gmet_list)
ds = ds.xvec.zonal_stats(geometry=in_geom.geometry, x_coords="lon", y_coords="lat", stats="mean", method="iterate")

# Add coords
## Make and reorder the location ID coordinates
loc_df = pd.DataFrame(in_geom[gdf_index_col]).set_index(in_geom.geometry)
loc_df = loc_df.reindex(in_geom.geometry.values)

## Calculate and reorder lat coordinates
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom.geometry)
lat_df = lat_df.reindex(in_geom.geometry.values)

xds = xr.Dataset(
    {
        "max_temp": (['location', 'time'], ds['daily_maximum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['location', 'time'], ds['daily_minimum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "location": (['location'], list(loc_df.iloc[:,0]), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": ds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

if 'pr' in GRIDMET_PARAMS:
    output = xr.merge([xds, vol_xds])
else: 
    output = xds

output


### Updated and archived get_gridmet_at_points()

In [ ]:
# get_gridmet_at_points()
in_geom = exres
gdf_index_col = "gageID"
start='2016-01-01'
end='2016-01-05'
crs = 4326

# overwrite configuration to run initial approch for pulling gridmet data and performing zonal stats
GRIDMET_PARAMS = ['tmmn', 'tmmx']
VOL_PARAMS = ['pr']


if gdf_index_col is not None:
    ixcol = gdf_index_col
else:
    in_geom['ixcol'] = in_geom.index
    ixcol = 'ixcol'

location_ids = in_geom[ixcol].to_list()

if (in_geom.geometry.geom_type == 'Point').all():
    coords = list(zip(in_geom.geometry.x, in_geom.geometry.y))
elif (in_geom.geometry.geom_type == 'Polygon').all():
    coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))
else:
    coords = None
    raise ValueError("Mixed geometry types were found in the input GeoDataFrame. Mixed Geometry is not supported.")

loc_lat = []
loc_lon = []
loc_elev = py3dep.elevation_bycoords(coords, crs=crs)  # only 4326 or NAD83 works with py3dep

if isinstance(loc_elev, list):
    loc_elev = loc_elev
else:
    loc_elev = [loc_elev]

loc_gdf = in_geom[['{0}'.format(ixcol), 'geometry']]

print("Retrieving GridMET cells...")
gmt_cells = get_gridmet_cells(loc_gdf)
unq_cells = gmt_cells['cell_id'].unique()
print("{0} unique GridMET cells found for {1} input features.".format(len(unq_cells), len(loc_gdf[ixcol])))

gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid

# Parameters retrieved to be averaged over watershed area here
tmmn = []
tmmx = []

cdsets = {}
print("Fetching GridMET data for unique cells...")
for cell in tqdm(unq_cells, desc='Cells'):
    clon = gmt_cntrs[cell].x
    clat = gmt_cntrs[cell].y
    datasets = []
    for p in GRIDMET_PARAMS:
        s = start
        e = end
        ds = GridMet(p, start=s, end=e, lat=clat, lon=clon).get_point_timeseries()
        datasets.append(ds)
    cdsets[cell] = datasets

# Parameters retried to be converted wot weighted volumes here
if len(VOL_PARAMS)> 1:
    raise ValueError("GRIDtools.grid_area_weighted_volume() is only compatible with the precip parameter")

# volparam_list = []
for p in VOL_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable= p, start=start, end=end, bbox=BBox(bnds[0]-0.5, bnds[2]+0.5, bnds[3]+0.5, bnds[1]-0.5))

    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, 'gageID')
    # volparam_list.append(vol_xds)
# xr.merge(volparam_list)

for i in range(len(coords)):
    c = coords[i]
    loc = location_ids[i]
    gmtcell_ids = gmt_cells[gmt_cells[ixcol] == loc]
    lon, lat = c
    loc_lat.append(lat)
    loc_lon.append(lon)


    if len(gmtcell_ids.index) > 1:

        tmmnm = []
        tmmxm = []

        for cid in gmtcell_ids['cell_id']:
            dset = cdsets[cid]

            tmmnm.append(dset[GRIDMET_PARAMS.index('tmmn')])
            tmmxm.append(dset[GRIDMET_PARAMS.index('tmmx')])

        tmmnm_d = pd.concat(tmmnm)
        tmmxm_d = pd.concat(tmmxm)

        tmmn.append(tmmnm_d.groupby(tmmnm_d.index).mean())
        tmmx.append(tmmxm_d.groupby(tmmxm_d.index).mean())

    else:
        dset = cdsets[gmtcell_ids['cell_id'].values[0]]
        tmmn.append(dset[GRIDMET_PARAMS.index('tmmn')])
        tmmx.append(dset[GRIDMET_PARAMS.index('tmmx')])

mean_xds = xr.Dataset(
    {
        "min_temp": (['time', 'location'], pd.concat(tmmn, axis=1), {'standard_name': 'Minimum Temperature',
                                                                    'units': 'Kelvin'}),
        "max_temp": (['time', 'location'], pd.concat(tmmx, axis=1), {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], loc_lat, {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "lon": (['location'], loc_lon, {'standard_name': 'longitude',
                                        'long_name': 'location_longitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "elev": (['location'], loc_elev, {'standard_name': 'elevation',
                                        'long_name': 'location_elevation',
                                        'units': 'meters'}),
        "location": (['location'], location_ids, {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": tmmn[0].index
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

xr.merge([mean_xds, vol_xds])